In [2]:
from scipy.stats import hypergeom
import statsmodels.stats.multitest as multi
import pandas as pd
import os
import glob
import requests


In [13]:
degfiles = ['../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_her19_v_herm_lfc1.txt',
            '../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_male_v_her19_lfc1.txt',
            '../../../her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_de_results_male_v_herm_lfc1.txt']

In [14]:
annfile = '../../../her7/02_functional_annotations/eggnog_annotations.txt'
outfile = '../../../her7/11_kegg_enrichment/functional_enrichment_results_KEGG_wminexp_lfc1.txt'
keggfile = '../../../../dbs/kegg/KEGG_KO_ALL_PARENTS_Jun-30-2024.txt'

In [15]:
# min/max thresholds for tests
highCount = 10000000
lowCount = 5

min_exp_thres = 3
de_lfc_thres = 1
de_padj_thres = 0.1

# Initialize set dictionaries

In [16]:
degsets = {}
expset = set()

for degfile in degfiles:
    #print(degfile)
    comparison = degfile.split('burow_de_results_')[1].split('_lfc1.txt')[0]
    #print(comparison)
    degsets[comparison + '_up'] = set()
    degsets[comparison + '_down'] = set()
    
    df = pd.read_csv(degfile, sep='\t', index_col=0)
    df.index.name = 'gene_id'
    #df
    
    # Filter rows based on conditions
    exp_df = df[(df['baseMean'] >= min_exp_thres)]
    de_up_df = df[(df['log2FoldChange'] > de_lfc_thres) & (df['padj'] < de_padj_thres)]
    de_down_df = df[(df['log2FoldChange'] < de_lfc_thres * -1) & (df['padj'] < de_padj_thres)]
    #print(de_up_df.loc[de_up_df['baseMean'].idxmin()])
    #print(de_down_df.loc[de_down_df['baseMean'].idxmin()])

    # Extract gene_id column as Python list
    for gene in exp_df.index.tolist():
        #print(gene)
        expset.add(gene)

    for gene in de_up_df.index.tolist():
        #print(gene)
        degsets[comparison + '_up'].add(gene)
        
    for gene in de_down_df.index.tolist():
        #print(gene)
        degsets[comparison + '_down'].add(gene)


In [17]:
# genes in Ceratopteris with min baseMean >= threshold (3)
len(expset)

29839

In [18]:
# total genes in Ceratopteris
! grep -v baseMean {degfile} | wc -l

36857


In [19]:
# initialize remaining data structures
isaDict = {}
keggDesc = {}
keggOrder = {}

codeDict = {}
codeDict['total'] = {}
lociDict = {}
lociDict['total'] = set()

print('Number of genes in each DEG set:')
for degset in degsets:
    print(degset, len(degsets[degset]))
    codeDict[degset] = {}
    lociDict[degset] = set()

Number of genes in each DEG set:
her19_v_herm_up 44
her19_v_herm_down 1422
male_v_her19_up 4727
male_v_her19_down 3754
male_v_herm_up 3464
male_v_herm_down 3087


# Load KEGG relationships

In [20]:
#! head {keggfile}

In [21]:
fi = open(keggfile)

for line in fi:
    currentParents = set()
    line = line.rstrip().split('\t')
    currentID = line.pop(0).split(':')[0]
    #print(currentID, line)
    
    for parent in line:
        currentParents.add(parent)
        
    #print(currentID, currentParents)
    if len(currentParents) == 0:
        continue
    else:
        isaDict[currentID] = currentParents

fi.close()

descfiles = os.path.realpath(keggfile)
descfiles = os.path.split(descfiles)[0] + '/*DESC_Jun-30-2024.txt'

for infile in glob.glob(descfiles):
    #print(infile)
    fi = open(infile)
    
    for line in fi:
        term, desc = line.rstrip().split('\t')
        #print(term, desc)
        keggDesc[term] = desc
    
    fi.close()


In [22]:
brite_url = 'http://rest.kegg.jp/get/br:br08901'
resp = requests.get(brite_url)

catA = ''
catB = ''

order = 0

for line in resp.text.split('\n'):
    order += 1
    cat = ''
    
    if len(line) == 0:
        continue
        
    if line[0] == '+' or line[0] == '!' or line[0] == '#':
        continue
    #print(line)
    
    if line[0] == 'A':
        catA = line[1:]
        cat = catA

    if catA == 'Organismal Systems' or catA == 'Human Diseases' or catA == 'Drug Development':
        continue

    if line[0] == 'B':
        catB = line[3:]
        cat = catB
        #print(catB)
        
    if catB == 'Global and overview maps':
        continue
    
    if line[0] == 'C':
        cat = 'map' + line[5:10]

    
    #print(cat, order)
    keggOrder[cat] = order


# Parse keggs in annotation file

In [23]:
#! head -6 {annfile}

In [24]:
fi = open(annfile)

for line in fi:
    if line[0] == '#':
        continue
        
    col = line.rstrip().split('\t')
    locus = 'Ceric.' + col[0].split('.')[1]
    
    # skip if locus is not expressed
    if locus not in expset:
        #print('skipping ', locus, ' not expressed')
        continue
    
    returnedparents = ''
        
    keggcol = col[11].split(',')
    #print(locus, keggcol)
    
    if keggcol == ['-']:
        continue
    keggSet = set()
    
    for kegg in keggcol:
        kegg = kegg.split(':')[1]
        #print(kegg)

        if kegg in isaDict:
            returnedparents = isaDict[kegg]

            for parent in returnedparents:
                if parent not in keggOrder:
                    continue
                #skip if parent is a Pathway and not a higher level category
                #if parent.startswith('map'):
                    #continue

                keggSet.add(parent)

    #print(locus, keggSet)
    
    for kegg in keggSet:
        #print(kegg)
        if kegg not in codeDict['total']:
            codeDict['total'][kegg] = set()

        codeDict['total'][kegg].add(locus)
        lociDict['total'].add(locus)    

        for degset in degsets:
            #print(degset)
            if locus in degsets[degset]:
    
                if kegg not in codeDict[degset]:
                    codeDict[degset][kegg] = set()

                codeDict[degset][kegg].add(locus)
                lociDict[degset].add(locus)

fi.close()

In [25]:
####################################
### Perform hypergeometric tests ###
####################################

df = pd.DataFrame(columns=['set','kegg','order','x','N','n','M','pval','genelist','kegglist'])
kegglist = ''

for degset in degsets:
    for kegg in codeDict[degset]:

        count = len(codeDict[degset][kegg])
        if count > highCount or count < lowCount:
            continue

        # x is the number of drawn "successes" (ie no. genes in degset and in kegg category)
        x = len(codeDict[degset][kegg])
        genelist = ', '.join(codeDict[degset][kegg])

        # N is the sample size (ie no. genes in degset)
        N = len(lociDict[degset])

        # n is the number of successes in the population (ie no. genes in kegg category [skipping genes not expressed])
        n = len(codeDict['total'][kegg])

        # M is the population size (ie no. genes total in any kegg category [skipping genes not expressed])
        M = len(lociDict['total'])

        # https://alexlenail.medium.com/understanding-and-implementing-the-hypergeometric-test-in-python-a7db688a7458
        # https://github.com/jdrudolph/goenrich
        pval = hypergeom.sf(x-1, M, n, N)

        order = keggOrder[kegg]

        if kegg in keggDesc:
            desc = keggDesc[kegg]
            kegg = kegg + ' - ' + desc

        df.loc[len(df.index)] = [degset,kegg,order,x,N,n,M,pval,genelist,kegglist]  


In [26]:
#########################################
### Adjust pvalues for multiple tests ###
#########################################
if len(df['pval'].tolist()) > 0:
    adjpval = multi.multipletests(df['pval'].tolist(), alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    df['adjpval'] = adjpval
    df['seqfreq'] = df['x'] / df['N']
    df['totalfreq'] = df['n'] / df['M']

    df = df[['set','kegg','order','x','N','seqfreq','n','M','totalfreq','pval','adjpval','genelist','kegglist']]

    df.to_csv(outfile, sep='\t', index=False)
